In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
location="datasets/Kickstartercombo.csv"
df=pd.read_csv(location)
df.head()

,Unnamed: 0,ID,name,main_category,category,country,state,goal,pledged,currency,launched,deadline,backers,usd pledged,usd_goal_real,usd_pledged_real
0,0,1000002330,The Songs of Adelaide & Abullah,Publishing,Poetry,GB,failed,1000.0,0.0,GBP,2015-08-11 12:12:00,2015-10-09 11:36:00,0,0.0,NaN,NaN
1,1,1000004038,Where is Hank?,Film & Video,Narrative Film,US,failed,45000.0,220.0,USD,2013-01-12 00:20:00,2013-02-26 00:20:00,3,220.0,NaN,NaN
2,2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,US,failed,5000.0,1.0,USD,2012-03-17 03:24:00,2012-04-16 04:24:00,1,1.0,NaN,NaN
3,3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,US,canceled,19500.0,1283.0,USD,2015-07-04 08:35:00,2015-08-29 01:00:00,14,1283.0,NaN,NaN
4,4,1000014025,Monarch Espresso Bar,Food,Restaurants,US,successful,50000.0,52375.0,USD,2016-02-26 13:38:00,2016-04-01 13:38:00,224,52375.0,NaN,NaN


In [3]:
kickset=df[['name','main_category','country','state','goal','pledged','launched','deadline','backers']]
kickset.head()



,name,main_category,country,state,goal,pledged,launched,deadline,backers
0,The Songs of Adelaide & Abullah,Publishing,GB,failed,1000.0,0.0,2015-08-11 12:12:00,2015-10-09 11:36:00,0
1,Where is Hank?,Film & Video,US,failed,45000.0,220.0,2013-01-12 00:20:00,2013-02-26 00:20:00,3
2,ToshiCapital Rekordz Needs Help to Complete Album,Music,US,failed,5000.0,1.0,2012-03-17 03:24:00,2012-04-16 04:24:00,1
3,Community Film Project: The Art of Neighborhoo...,Film & Video,US,canceled,19500.0,1283.0,2015-07-04 08:35:00,2015-08-29 01:00:00,14
4,Monarch Espresso Bar,Food,US,successful,50000.0,52375.0,2016-02-26 13:38:00,2016-04-01 13:38:00,224


In [4]:
kickset.isnull().sum()

name             8
main_category    0
country          0
state            0
goal             0
pledged          0
launched         0
deadline         0
backers          0
dtype: int64

In [5]:
kicksetclean=kickset.dropna(subset=['name'])

In [6]:
kicksetclean.isnull().sum()

name             0
main_category    0
country          0
state            0
goal             0
pledged          0
launched         0
deadline         0
backers          0
dtype: int64

In [7]:
kicksetclean['state']=kicksetclean['state'].replace('canceled','failed')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
kicksetclean.head(10)

,name,main_category,country,state,goal,pledged,launched,deadline,backers
0,The Songs of Adelaide & Abullah,Publishing,GB,failed,1000.0,0.00,2015-08-11 12:12:00,2015-10-09 11:36:00,0
1,Where is Hank?,Film & Video,US,failed,45000.0,220.00,2013-01-12 00:20:00,2013-02-26 00:20:00,3
2,ToshiCapital Rekordz Needs Help to Complete Album,Music,US,failed,5000.0,1.00,2012-03-17 03:24:00,2012-04-16 04:24:00,1
3,Community Film Project: The Art of Neighborhoo...,Film & Video,US,failed,19500.0,1283.00,2015-07-04 08:35:00,2015-08-29 01:00:00,14
4,Monarch Espresso Bar,Food,US,successful,50000.0,52375.00,2016-02-26 13:38:00,2016-04-01 13:38:00,224
5,Support Solar Roasted Coffee & Green Energy! ...,Food,US,successful,1000.0,1205.00,2014-12-01 18:30:00,2014-12-21 18:30:00,16
6,Chaser Strips. Our Strips make Shots their B*tch!,Food,US,failed,25000.0,453.00,2016-02-01 20:05:00,2016-03-17 19:05:00,40
7,SPIN - Premium Retractable In-Ear Headphones w...,Design,US,failed,125000.0,8233.00,2014-04-24 18:14:00,2014-05-29 18:14:00,58
8,STUDIO IN THE SKY - A Documentary Feature Film...,Film & Video,US,failed,65000.0,6240.57,2014-07-11 21:55:00,2014-08-10 21:55:00,43
9,Of Jesus and Madmen,Publishing,CA,failed,2500.0,0.00,2013-09-09 18:19:00,2013-10-09 18:19:00,0


In [9]:
kicksetclean['state'].unique()

array(['failed', 'successful', 'live', 'undefined', 'suspended'], dtype=object)

In [10]:
kicksetclean=kicksetclean[kicksetclean.state != 'undefined']

In [11]:
kicksetclean=kicksetclean[kicksetclean.state != 'suspended']

In [12]:
kicksetclean['state'].unique()

array(['failed', 'successful', 'live'], dtype=object)

In [13]:
kicksetlive=kicksetclean[kicksetclean['state']=='live']

In [14]:
kicksetlive['state'].unique()

array(['live'], dtype=object)

In [15]:
kicksetclean=kicksetclean[kicksetclean.state != 'live']

In [16]:
kicksetclean['state'].unique()

array(['failed', 'successful'], dtype=object)

In [17]:
def difference(x, a, b):
    x=kicksetclean['pledged']
    for n in x:
        return a-b

            
kicksetclean['excess']=difference(kicksetclean['pledged'], kicksetclean['pledged'], kicksetclean['goal'])
kicksetclean.head()

,name,main_category,country,state,goal,pledged,launched,deadline,backers,excess
0,The Songs of Adelaide & Abullah,Publishing,GB,failed,1000.0,0.0,2015-08-11 12:12:00,2015-10-09 11:36:00,0,-1000.0
1,Where is Hank?,Film & Video,US,failed,45000.0,220.0,2013-01-12 00:20:00,2013-02-26 00:20:00,3,-44780.0
2,ToshiCapital Rekordz Needs Help to Complete Album,Music,US,failed,5000.0,1.0,2012-03-17 03:24:00,2012-04-16 04:24:00,1,-4999.0
3,Community Film Project: The Art of Neighborhoo...,Film & Video,US,failed,19500.0,1283.0,2015-07-04 08:35:00,2015-08-29 01:00:00,14,-18217.0
4,Monarch Espresso Bar,Food,US,successful,50000.0,52375.0,2016-02-26 13:38:00,2016-04-01 13:38:00,224,2375.0


In [18]:
kicksetclean['launched']=pd.to_datetime(kicksetclean['launched'])

In [19]:
kicksetclean['deadline']=pd.to_datetime(kicksetclean['deadline'])

In [20]:
kicksetclean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 684718 entries, 0 to 702410
Data columns (total 10 columns):
name             684718 non-null object
main_category    684718 non-null object
country          684718 non-null object
state            684718 non-null object
goal             684718 non-null float64
pledged          684718 non-null float64
launched         684718 non-null datetime64[ns]
deadline         684718 non-null datetime64[ns]
backers          684718 non-null int64
excess           684718 non-null float64
dtypes: datetime64[ns](2), float64(3), int64(1), object(4)
memory usage: 57.5+ MB


In [21]:
def difference2(x, a, b):
    x=kicksetclean['deadline']
    for n in x:
        return a-b

            
kicksetclean['interval']=difference2(kicksetclean['deadline'], kicksetclean['deadline'], kicksetclean['launched'])
kicksetclean.head()

,name,main_category,country,state,goal,pledged,launched,deadline,backers,excess,interval
0,The Songs of Adelaide & Abullah,Publishing,GB,failed,1000.0,0.0,2015-08-11 12:12:00,2015-10-09 11:36:00,0,-1000.0,58 days 23:24:00
1,Where is Hank?,Film & Video,US,failed,45000.0,220.0,2013-01-12 00:20:00,2013-02-26 00:20:00,3,-44780.0,45 days 00:00:00
2,ToshiCapital Rekordz Needs Help to Complete Album,Music,US,failed,5000.0,1.0,2012-03-17 03:24:00,2012-04-16 04:24:00,1,-4999.0,30 days 01:00:00
3,Community Film Project: The Art of Neighborhoo...,Film & Video,US,failed,19500.0,1283.0,2015-07-04 08:35:00,2015-08-29 01:00:00,14,-18217.0,55 days 16:25:00
4,Monarch Espresso Bar,Food,US,successful,50000.0,52375.0,2016-02-26 13:38:00,2016-04-01 13:38:00,224,2375.0,35 days 00:00:00


In [78]:
kicksetlive['launched']=pd.to_datetime(kicksetlive['launched'])

In [79]:
kicksetlive['deadline']=pd.to_datetime(kicksetlive['deadline'])

In [80]:
def difference3(x, a, b):
    x=kicksetlive['deadline']
    for n in x:
        return a-b

            
kicksetlive['interval']=difference3(kicksetlive['deadline'], kicksetlive['deadline'], kicksetlive['launched'])
kicksetlive.head()

,name,main_category,country,state,goal,pledged,launched,deadline,backers,interval
92,Disaster Area: A Podcast,Publishing,US,live,1000.0,125.0,2016-11-28 17:58:00,2016-12-28 17:58:00,3,30 days 00:00:00
129,#NotMyPresident Protest Tees,Fashion,US,live,2500.0,25.0,2016-11-22 04:46:00,2016-12-22 04:46:00,1,30 days 00:00:00
174,APP - CUSTOM SHOES/ Calzado Personalizado - MA...,Technology,MX,live,55000.0,1700.0,2016-11-19 04:05:00,2016-12-21 06:59:00,7,32 days 02:54:00
179,Strawberry Fields Café and Patisserie,Food,CA,live,50000.0,0.0,2016-12-01 21:16:00,2017-01-23 17:14:00,0,52 days 19:58:00
187,CLOTHING LABEL WITH A 90's STATE OF MIND - MAD...,Fashion,FR,live,3286.0,2866.0,2016-11-18 18:51:00,2016-12-18 18:51:00,32,30 days 00:00:00


In [22]:
kicksetclean.corr()

,goal,pledged,backers,excess
goal,1.000000,0.007372,0.004673,-0.996658
pledged,0.007372,1.000000,0.730397,0.074340
backers,0.004673,0.730397,1.000000,0.055007
excess,-0.996658,0.074340,0.055007,1.000000


In [81]:
def difference4(x, a, b):
    x=kicksetlive['pledged']
    for n in x:
        return a-b

            
kicksetlive['excess']=difference(kicksetlive['pledged'], kicksetlive['pledged'], kicksetlive['goal'])
kicksetlive.head()

,name,main_category,country,state,goal,pledged,launched,deadline,backers,interval,excess
92,Disaster Area: A Podcast,Publishing,US,live,1000.0,125.0,2016-11-28 17:58:00,2016-12-28 17:58:00,3,30 days 00:00:00,-875.0
129,#NotMyPresident Protest Tees,Fashion,US,live,2500.0,25.0,2016-11-22 04:46:00,2016-12-22 04:46:00,1,30 days 00:00:00,-2475.0
174,APP - CUSTOM SHOES/ Calzado Personalizado - MA...,Technology,MX,live,55000.0,1700.0,2016-11-19 04:05:00,2016-12-21 06:59:00,7,32 days 02:54:00,-53300.0
179,Strawberry Fields Café and Patisserie,Food,CA,live,50000.0,0.0,2016-12-01 21:16:00,2017-01-23 17:14:00,0,52 days 19:58:00,-50000.0
187,CLOTHING LABEL WITH A 90's STATE OF MIND - MAD...,Fashion,FR,live,3286.0,2866.0,2016-11-18 18:51:00,2016-12-18 18:51:00,32,30 days 00:00:00,-420.0


In [122]:
modeldf=pd.get_dummies(data=kicksetclean, columns=['state','main_category', 'country'])
modeldf.head()

,name,goal,pledged,launched,deadline,backers,excess,interval,state_failed,state_successful,...,country_LU,country_MX,"country_N,""0","country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
0,The Songs of Adelaide & Abullah,1000.0,0.0,2015-08-11 12:12:00,2015-10-09 11:36:00,0,-1000.0,58 days 23:24:00,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Where is Hank?,45000.0,220.0,2013-01-12 00:20:00,2013-02-26 00:20:00,3,-44780.0,45 days 00:00:00,1,0,...,0,0,0,0,0,0,0,0,0,1
2,ToshiCapital Rekordz Needs Help to Complete Album,5000.0,1.0,2012-03-17 03:24:00,2012-04-16 04:24:00,1,-4999.0,30 days 01:00:00,1,0,...,0,0,0,0,0,0,0,0,0,1
3,Community Film Project: The Art of Neighborhoo...,19500.0,1283.0,2015-07-04 08:35:00,2015-08-29 01:00:00,14,-18217.0,55 days 16:25:00,1,0,...,0,0,0,0,0,0,0,0,0,1
4,Monarch Espresso Bar,50000.0,52375.0,2016-02-26 13:38:00,2016-04-01 13:38:00,224,2375.0,35 days 00:00:00,0,1,...,0,0,0,0,0,0,0,0,0,1


In [126]:
y= modeldf['state_successful']

In [127]:
X = modeldf.drop(['state_successful','state_failed', 'name', 'launched', 'deadline', 'interval','excess'], axis=1)

In [128]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [129]:
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [130]:
LogReg.score(X_train, y_train)

0.99395918754814938

In [131]:
y_pred=LogReg.predict(X_test)

In [132]:
cm=pd.DataFrame(confusion_matrix(y_test, y_pred), columns=['successful', 'failed'], index=['True successful', 'True failed'])

In [133]:
cm

,successful,failed
True successful,86717,827
True failed,0,49400


In [134]:
LogReg.score(X_test, y_test)

0.99396103516765977

In [135]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       1.00      0.99      1.00     87544
          1       0.98      1.00      0.99     49400

avg / total       0.99      0.99      0.99    136944



In [136]:
modeldf.corr()

,goal,pledged,backers,excess,state_failed,state_successful,main_category_Art,main_category_Comics,main_category_Crafts,main_category_Dance,...,country_LU,country_MX,"country_N,""0","country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
goal,1.000000,0.007372,0.004673,-0.996658,0.024961,-0.024961,-0.002178,-0.003919,-0.004870,-0.003397,...,-0.000208,0.010157,-0.000436,-0.000436,0.004184,0.018047,-0.000749,0.016538,-0.000653,-0.008823
pledged,0.007372,1.000000,0.730397,0.074340,-0.109906,0.109906,-0.017902,-0.004343,-0.012195,-0.006231,...,-0.000458,0.009402,0.001013,0.001013,-0.001680,0.008944,-0.001579,0.034099,0.001790,0.002259
backers,0.004673,0.730397,1.000000,0.055007,-0.124657,0.124657,-0.019639,0.005586,-0.012773,-0.006784,...,-0.000773,-0.004236,-0.002083,-0.002083,-0.003310,-0.003120,-0.002471,-0.001234,0.000947,0.012794
excess,-0.996658,0.074340,0.055007,1.000000,-0.033871,0.033871,0.000710,0.003554,0.003860,0.002878,...,0.000170,-0.009361,0.000518,0.000518,-0.004310,-0.017267,0.000618,-0.013708,0.000798,0.008983
state_failed,0.024961,-0.109906,-0.124657,-0.033871,1.000000,-1.000000,-0.029654,-0.061791,0.037907,-0.056866,...,0.000683,0.012223,-0.003509,-0.003509,0.027760,0.011846,0.006836,0.010006,0.000717,-0.066138
state_successful,-0.024961,0.109906,0.124657,0.033871,-1.000000,1.000000,0.029654,0.061791,-0.037907,0.056866,...,-0.000683,-0.012223,0.003509,0.003509,-0.027760,-0.011846,-0.006836,-0.010006,-0.000717,0.066138
main_category_Art,-0.002178,-0.017902,-0.019639,0.000710,-0.029654,0.029654,1.000000,-0.048650,-0.043647,-0.029143,...,0.000361,0.003604,-0.005253,-0.005253,-0.002455,-0.003876,-0.002892,-0.001851,-0.003598,0.008472
main_category_Comics,-0.003919,-0.004343,0.005586,0.003554,-0.061791,0.061791,-0.048650,1.000000,-0.026077,-0.017412,...,-0.000530,-0.002257,-0.003139,-0.003139,-0.012041,-0.001237,-0.006323,-0.005513,-0.002374,0.017014
main_category_Crafts,-0.004870,-0.012195,-0.012773,0.003860,0.037907,-0.037907,-0.043647,-0.026077,1.000000,-0.015621,...,-0.001811,0.004253,-0.002816,-0.002816,-0.000045,0.000138,0.001947,-0.000157,0.005366,-0.010321
main_category_Dance,-0.003397,-0.006231,-0.006784,0.002878,-0.056866,0.056866,-0.029143,-0.017412,-0.015621,1.000000,...,-0.001209,0.001766,-0.001880,-0.001880,-0.003673,0.000215,-0.002415,-0.004265,-0.002539,0.017855


In [ ]:
writer = pd.ExcelWriter('group3kick.xlsx',engine='xlsxwriter')
kicksetclean.to_excel(writer, sheet_name='Sheet1')
writer.save()

**Test model on live campaign data**

In [97]:
modeldf2=pd.get_dummies(data=kicksetlive, columns=['main_category', 'country'])
modeldf2.head()

,name,state,goal,pledged,launched,deadline,backers,interval,excess,main_category_Art,...,country_LU,country_MX,"country_N,""0","country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
92,Disaster Area: A Podcast,live,1000.0,125.0,2016-11-28 17:58:00,2016-12-28 17:58:00,3,30 days 00:00:00,-875.0,0,...,0,0,0,0,0,0,0,0,0,1
129,#NotMyPresident Protest Tees,live,2500.0,25.0,2016-11-22 04:46:00,2016-12-22 04:46:00,1,30 days 00:00:00,-2475.0,0,...,0,0,0,0,0,0,0,0,0,1
174,APP - CUSTOM SHOES/ Calzado Personalizado - MA...,live,55000.0,1700.0,2016-11-19 04:05:00,2016-12-21 06:59:00,7,32 days 02:54:00,-53300.0,0,...,0,1,0,0,0,0,0,0,0,0
179,Strawberry Fields Café and Patisserie,live,50000.0,0.0,2016-12-01 21:16:00,2017-01-23 17:14:00,0,52 days 19:58:00,-50000.0,0,...,0,0,0,0,0,0,0,0,0,0
187,CLOTHING LABEL WITH A 90's STATE OF MIND - MAD...,live,3286.0,2866.0,2016-11-18 18:51:00,2016-12-18 18:51:00,32,30 days 00:00:00,-420.0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
w= modeldf2['state']

In [101]:
Z = modeldf2.drop(['state', 'name', 'launched', 'deadline'], axis=1)
Z.head()

,goal,pledged,backers,interval,excess,main_category_Art,main_category_Comics,main_category_Crafts,main_category_Dance,main_category_Design,...,country_LU,country_MX,"country_N,""0","country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
92,1000.0,125.0,3,30 days 00:00:00,-875.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
129,2500.0,25.0,1,30 days 00:00:00,-2475.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
174,55000.0,1700.0,7,32 days 02:54:00,-53300.0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
179,50000.0,0.0,0,52 days 19:58:00,-50000.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
187,3286.0,2866.0,32,30 days 00:00:00,-420.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
Z_train, Z_test, w_train, w_test = train_test_split(Z, w, test_size=0.2, random_state=5)

In [103]:
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [52]:
LogReg.score(X_train, y_train)

0.9967176244217506

In [107]:
w_pred=LogReg.predict(Z_test)

TypeError: float() argument must be a string or a number, not 'Timedelta'

In [108]:
cm=pd.DataFrame(confusion_matrix(Z_test, Z_pred), columns=['successful', 'failed'], index=['True successful', 'True failed'])

NameError: name 'Z_pred' is not defined

In [55]:
cm

,successful,failed
True successful,87063,481
True failed,0,49400


In [56]:
LogReg.score(X_test, y_test)

0.99648761537562802

In [57]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

     failed       1.00      0.99      1.00     87544
 successful       0.99      1.00      1.00     49400

avg / total       1.00      1.00      1.00    136944

